# Initialization

In [22]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))


;

In [31]:
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [4]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [[33.0 2 "Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 [33.0 2 "Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 [33.0 2 "Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 [33.0 2 "Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 [33.0 2 "Feeder_450" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

# services = [[33.0 2 "Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
#             [33.0 2 "Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]

services = [
    [110000 2 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [100000 2 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [90000 2 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [970000 2 "Feeder_450" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]


# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [5]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [6]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end


edges = vcat(s_t_adj, transshipment_adj, ports_adj, penalty_adj, dummy_adj);

In [7]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

In [8]:
function dijkstra(G, source, destination) # HUSK DUAL VARIABLES
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

## Flowproblem

### Initialization

In [65]:
paths = []
costs = []
;

In [66]:
# Decision variables
decision_var = []
for edge in edges
    push!(decision_var, string(edge[1], "|", edge[2]))
end

# Initializing parameters
dual_var = zeros(length(decision_var))

# Updated edges w.r.t. dual costs
edges_dual = edges
for i in 1:length(edges_dual)
    edges_dual[i][3] = edges_dual[i][3] + dual_var[i]
end

# Paths and costs from shortest path
for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(paths, path)
    push!(costs, cost)
end

for i in penalty_adj
    push!(paths, i[1:2])
    push!(costs, i[3])
end

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
k = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
s = 1:n_s

n_p = size(paths, 1) # Number of paths
p = 1:n_p

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff

ij = size(decision_var, 1) # Number of edges
;

In [67]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, ij, 0) # vessel capacity if service s uses edge (i,j), else 0
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path

for i in 1:length(paths)
    ak = zeros(Float64, ij)
    for j in 2:length(paths[i])
        ak[findall(x -> x == string(paths[i][j-1], "|", paths[i][j]), decision_var)[1]] = 1
    end
    aij = [aij ak]
end

for i in 1:length(services_w_transshipment)
    us = zeros(Float64, ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), decision_var)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), decision_var)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), decision_var)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

for i in 1:length(paths)
    pk = zeros(Float64, n_k)
    pk[findall(x -> x == string(paths[i][1], "|", paths[i][end]), commodities)[1]] = 1
    PK = [PK pk]
end

### Actual problem

In [39]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[p] >= 0)
@variable(flowModel, y[s], Bin)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in s) + sum(costs[j]*x[j] for j in p))
@constraint(flowModel, [i=1:ij], sum(aij[i, j] * x[j] for j in p) <= sum(uij[i, j] * y[j] for j in s))
@constraint(flowModel, [i=1:n_k], x[i] == baltic_demand.FFEPerWeek[i]) # Vi skal inkludere 
print(flowModel)
#optimize!(flowModel)
#solution_summary(flowModel)

#objective_value(flowModel)
#has_duals(flowModel)

Min 110000 y[1] + 100000 y[2] + 90000 y[3] + 970000 y[4] + 409 x[1] + 642 x[2] + 528 x[3] + 446 x[4] + 897 x[5] + 297 x[6] + 473 x[7] + 578 x[8] + 446 x[9] + 409 x[10] + 354 x[11] + 578 x[12] + 350 x[13] + 897 x[14] + 297 x[15] + 473 x[16] + 528 x[17] + 350 x[18] + 642 x[19] + 483 x[20] + 354 x[21] + 483 x[22] + 1000 x[23] + 1000 x[24] + 1000 x[25] + 1000 x[26] + 1000 x[27] + 1000 x[28] + 1000 x[29] + 1000 x[30] + 1000 x[31] + 1000 x[32] + 1000 x[33] + 1000 x[34] + 1000 x[35] + 1000 x[36] + 1000 x[37] + 1000 x[38] + 1000 x[39] + 1000 x[40] + 1000 x[41] + 1000 x[42] + 1000 x[43] + 1000 x[44]
Subject to
 

x[1] = 77
 x[2] = 456
 x[3] = 65
 x[4] = 7
 x[5] = 10
 x[6] = 98
 x[7] = 660
 x[8] = 17
 x[9] = 268
 x[10] = 18
 x[11] = 16
 x[12] = 37
 x[13] = 187
 x[14] = 50
 x[15] = 231
 x[16] = 597
 x[17] = 32
 x[18] = 162
 x[19] = 397
 x[20] = 1215
 x[21] = 6
 x[22] = 298
 x[3] + x[5] + x[8] - 450 y[1] ≤ 0
 x[12] + x[14] + x[17] - 450 y[1] ≤ 0
 x[6] + x[9] + x[10] + x[13] - 450 y[2] ≤ 0
 x[1] + x[4] + x[15] + x[18] - 450 y[2] ≤ 0
 x[2] + x[21] - 450 y[3] ≤ 0
 x[11] + x[19] - 450 y[3] ≤ 0
 x[16] + x[20] - 450 y[4] ≤ 0
 x[7] + x[22] - 450 y[4] ≤ 0
 x[17] - 450 y[1] ≤ 0
 x[3] - 450 y[1] ≤ 0
 x[12] - 450 y[1] ≤ 0
 x[8] - 450 y[1] ≤ 0
 x[14] - 450 y[1] ≤ 0
 x[5] - 450 y[1] ≤ 0
 x[4] - 450 y[2] ≤ 0
 x[9] - 450 y[2] ≤ 0
 x[18] - 450 y[2] ≤ 0
 x[13] - 450 y[2] ≤ 0
 x[1] - 450 y[2] ≤ 0
 x[10] - 450 y[2] ≤ 0
 x[15] - 450 y[2] ≤ 0
 x[6] - 450 y[2] ≤ 0
 x[19] - 450 y[3] ≤ 0
 x[2] - 450 y[3] ≤ 0
 -450 y[3] ≤ 0
 -450 y[3] ≤ 0
 x[7] - 450 y[4] ≤ 0
 x[16] - 450 y[4] ≤ 0
 x[11] - 450 y[3] ≤ 0
 x[21] - 450 y[3] ≤

Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.

In [130]:
flowModel = Model() # Model for the master problem
n_k = size(constraints, 1)
k = 1:size(constraints, 1) # Initial number of variables
A = Matrix(I, n_k, n_k)

@variable(flowModel, x[k] >= 0) # Defining the variables
@objective(flowModel, Min, sum(x[j] for j in k)) # Setting the objective
@constraint(flowModel, [i=1:n_k], sum(A[i, j] * x[j] for j in k) == 0)



1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1] = 0

In [ ]:
W=100 # skibe cap
cardinalityM=5
M=[1:cardinalityM]
A=Matrix(I, cardinalityM, cardinalityM)
p=zeros(5)
b=[45; 38; 25; 11; 12]
w=[22; 42; 52; 53; 78]

# flowModel = Model() # Model for the master problem
# Jprime = 1:size(A, 2) # Initial number of variables
# @variable(flowModel, 0 <= x[Jprime] <= 1000000) # Defining the variables
# @objective(flowModel, Min, sum(x[j] for j in Jprime)) # Setting the objective
# @constraint(flowModel, [i=1:cardinalityM], sum(A[i, j] * x[j] for j in Jprime) == b[i])
# print(flowModel)

flowModel = Model() # Model for the master problem
n_k = size(baltic_demand, 1)
k = 1:size(baltic_demand, 1) # Initial number of variables
A = Matrix(I, n_k, n_k)

@variable(flowModel, x[k] >= 0) # Defining the variables
@objective(flowModel, Min, sum(x[j] for j in k)) # Setting the objective
@constraint(flowModel, [constraints[i]], sum(A[i, j] * (x[j] + x[1]) for j in k, i in n_k) == 0)


# J = 1_000  # Some large number
# model = Model(HiGHS.Optimizer)
# set_silent(model)
# @variable(model, x[x_ij] >= 0, Int)
# @variable(model, y[getindex.(c_y, 1)] >= 0, Int)
# ;